### Data Preparation

In [126]:
import pandas as pd
import numpy as np
import folium
import osmnx
import networkx
import time

from smart_mobility_utilities.common import cost, Node, probability,randomized_search
from smart_mobility_utilities.viz import draw_route, draw_map
from smart_mobility_utilities.children import *

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# A fixed degree precision of E/W at 45N/S with 0.0001 degree.
CORD_TR = 7.87

# A fixed lane wide
LANE_WD = 3.33

# Alpha parameter
ALPHA = 0.172

In [102]:
# Retrieve traffice flow data for preprocessing.
df_traffic_flow = pd.read_csv("traffic-flow-2020.csv")

# Retrieve node and edge data. 
graph = osmnx.graph_from_address("Toronto", dist=3000, clean_periphery=True, simplify=True, network_type='drive_service')
nodes, edges = osmnx.graph_to_gdfs(graph)

In [103]:
nodes.head()

,y,x,ref,highway,street_count,geometry
osmid,,,,,,
3458665,43.648070,-79.360781,155,motorway_junction,3,POINT (-79.36078 43.64807)
3458680,43.647613,-79.361605,NaN,NaN,3,POINT (-79.36160 43.64761)
3458699,43.646231,-79.368173,154,motorway_junction,3,POINT (-79.36817 43.64623)
3458707,43.644044,-79.376439,NaN,NaN,3,POINT (-79.37644 43.64404)
3458730,43.640081,-79.385192,NaN,NaN,3,POINT (-79.38519 43.64008)


In [104]:
edges.head()

osmid  \
u       v          key                                                      
3458665 208539722  0                                [19882352, 179146498]   
        3458699    0                               [329385928, 863872068]   
3458680 2260363024 0    [98243535, 4018705, 216525622, 216525624, 4275...   
3458699 208539709  0                       [27751073, 34398524, 19882351]   
        3458707    0                                 [98243452, 98243455]   

                       bridge  oneway   lanes        highway reversed  \
u       v          key                                                  
3458665 208539722  0      yes    True       1  motorway_link    False   
        3458699    0      yes    True       4       motorway    False   
3458680 2260363024 0      yes    True  [2, 4]       motorway    False   
3458699 208539709  0      yes    True       2  motorway_link    False   
        3458707    0      yes    True  [2, 3]       motorway    False   

                          length  \
u       v          key             
3458665 208539722  0     465.454   
        3458699    0     629.858   
3458680 2260363024 0    1484.732   
3458699 208539709  0     380.611   
        3458707    0     711.523   

                                                                 geometry  \
u       v          key                                                      
3458665 208539722  0    LINESTRING (-79.36078 43.64807, -79.36130 43.6...   
        3458699    0    LINESTRING (-79.36078 43.64807, -79.36149 43.6...   
3458680 2260363024 0    LINESTRING (-79.36160 43.64761, -79.35794 43.6...   
3458699 208539709  0    LINESTRING (-79.36817 43.64623, -79.36871 43.6...   
        3458707    0    LINESTRING (-79.36817 43.64623, -79.36870 43.6...   

                                    ref  \
u       v          key                    
3458665 208539722  0                NaN   
        3458699    0           Gardiner   
3458680 2260363024 0    [DVP, Gardiner]   
3458699 208539709  0                NaN   
        3458707    0           Gardiner   

                                                             name maxspeed  \
u       v          key                                                       
3458665 208539722  0                                          NaN      NaN   
        3458699    0                          Gardiner Expressway       90   
3458680 2260363024 0    [Gardiner Expressway, Don Valley Parkway]       90   
3458699 208539709  0                                          NaN      NaN   
        3458707    0                          Gardiner Expressway       90   

                       width service access junction tunnel  
u       v          key                                       
3458665 208539722  0     NaN     NaN    NaN      NaN    NaN  
        3458699    0     NaN     NaN    NaN      NaN    NaN  
3458680 2260363024 0     NaN     NaN    NaN      NaN    NaN  
3458699 208539709  0     NaN     NaN    NaN      NaN    NaN  
        3458707    0     NaN     NaN    NaN      NaN    NaN

In [105]:
df_traffic_flow['count_date'] = pd.to_datetime(df_traffic_flow['count_date'], format='%Y-%m-%d')
df_flow = df_traffic_flow.groupby(['location_id']).mean().reset_index()
# Pedestrian, bike, and scoter doesn't matter. 
g_flow = df_flow.drop(columns=['nx_peds', 'sx_peds', 'ex_peds', 'wx_peds', 
                              'nx_bike', 'sx_bike', 'ex_bike', 'wx_bike', 
                              'nx_other', 'sx_other', 'ex_other', 'wx_other'])
g_flow = g_flow.loc[g_flow['centreline_type'] == 2]

# Add traffic flow together
n_previous = 35
g_flow["total"] = g_flow.iloc[:,-n_previous:].sum(axis=1)

# Normalize for reference only, will not be applied to edges. Weight of edges will be calculated individually. 
g_flow['n-total'] = (g_flow['total'] - g_flow['total'].min())/(g_flow['total'].max() - g_flow['total'].min())

g_flow.head()

,location_id,_id,count_id,lng,lat,centreline_type,centreline_id,px,sb_cars_r,sb_cars_t,...,nb_bus_t,nb_bus_l,wb_bus_r,wb_bus_t,wb_bus_l,eb_bus_r,eb_bus_t,eb_bus_l,total,n-total
1,3925,59795.5,43407.0,-79.475274,43.636780,2.0,13468657.0,NaN,0.00000,122.25000,...,0.78125,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,308.96875,0.259248
2,3926,91443.5,45387.0,-79.485752,43.648312,2.0,13467247.0,334.0,4.21875,7.90625,...,0.00000,0.21875,0.03125,0.796875,0.34375,0.171875,1.03125,0.015625,572.71875,0.482588
3,3934,4976.5,39943.0,-79.534016,43.594361,2.0,13470639.0,240.0,8.43750,0.53125,...,0.00000,0.15625,0.00000,1.125000,0.06250,0.000000,1.18750,0.000000,327.06250,0.274570
4,3936,14835.5,40587.0,-79.483267,43.620511,2.0,13469793.0,1994.0,3.87500,0.00000,...,0.00000,0.00000,0.71875,0.718750,0.00000,0.000000,1.50000,0.000000,291.78125,0.244694
5,3939,24115.5,41169.0,-79.482878,43.688368,2.0,13460466.0,1422.0,30.93750,157.65625,...,1.62500,0.03125,0.37500,4.062500,0.28125,0.031250,3.93750,0.062500,676.53125,0.570495


In [106]:
edges['traffic_flow'] = 0.0

In [107]:
nodes_fixed = nodes.reset_index()

In [108]:
def find_traffic_flow(osmid, nodes, g_flow, lanes):
    lat = float(nodes.loc[nodes['osmid'] == osmid].y)
    lng = float(nodes.loc[nodes['osmid'] == osmid].x)
    
    tr = 0.0001*((LANE_WD * lanes)/CORD_TR)
    
    result = g_flow.loc[(g_flow['lat'] >= (lat-tr)) & (g_flow['lat'] <= (lat+tr)) & (g_flow['lng'] >= (lng-tr)) & (g_flow['lng'] <= (lng+tr))]
    
    # If there has no traffic flow on given edge, we pretend it has a reletively small traffic.
    if len(result) != 0:
        return float(result['total'])
    else:
        return float(g_flow['total'].quantile(0.25))
    

Calculate traffic flow on each edge.

In [109]:
for i, row in edges.iterrows():
    u = i[0]
    v = i[1]   
    
    try:
        lanes_num = int(row.lanes[0])
    except IndexError and TypeError as e:
        if pd.isnull(row.lanes):
            lanes_num = 1
        else:
            lanes_num = int(row.lanes)
    
    flow_u = find_traffic_flow(u, nodes_fixed, g_flow, lanes_num)
    flow_v = find_traffic_flow(v, nodes_fixed, g_flow, lanes_num)
    total_flow = flow_u + flow_v
    
    edges.loc[i, 'traffic_flow'] = total_flow
    
    

Normalize traffice flow. 

In [110]:
edges['n_traffic_flow'] = (edges['traffic_flow'] - edges['traffic_flow'].min())/(edges['traffic_flow'].max() - edges['traffic_flow'].min())

To avoid 0 as divider.

In [111]:
edges.loc[edges['n_traffic_flow'] == 1, 'n_traffic_flow'] = 0.999

In [112]:
edges['updated_d'] = edges['length']/(1-edges['n_traffic_flow'])

Fill nan in maxspeed as 50 (A common speed limit on most roads)

In [113]:
edges[['maxspeed']] = edges[['maxspeed']].fillna(value=50)

In [123]:
edges['t'] = " "

for i, row in edges.iterrows():
    try:
        speed = float(row.maxspeed)
    except ValueError and TypeError as e:
        speed = float(row.maxspeed[0])
        
    if row.n_traffic_flow >= ALPHA:
        t = row.updated_d/speed
    else:
        t = row.length/speed
        
    edges.loc[i, 't'] = t
        

In [125]:
adjusted_g = osmnx.graph_from_gdfs(nodes, edges)

### Apply Dijkstra

Assume one collection center and two drop location.

In [127]:
collection = ['05b07a8a-6f40-43f7-aff9-0cc371e26dd8', 43.64348087, -79.39985672]
drop1 = ['9e550581-c823-4a16-9e7c-82cf5c94fa2f', 43.6406584, -79.3956929]
drop2 = ['4824a0da-7572-4988-8cd7-5559e5614393', 43.6357152, -79.3998268]

In [128]:
source_point = (collection[1], collection[2])
destination_point1 = (drop1[1], drop1[2])
destination_point2 = (drop2[1], drop2[2])

X_1 = [source_point[1], destination_point1[1]]
Y_1 = [source_point[0], destination_point1[0]]
closest_nodes_1 = osmnx.distance.nearest_nodes(graph,X_1,Y_1)

# Here, we use our weights 't' to calculate shortest path with dijkstra
start_1 = time.time()
shortest_route_1 = networkx.shortest_path(G=graph,source=closest_nodes_1[0],target=closest_nodes_1[1], weight='t', method='dijkstra')
stop_1 = time.time()

X_2 = [destination_point1[1], destination_point2[1]]
Y_2 = [destination_point1[0], destination_point2[0]]
closest_nodes_2 = osmnx.distance.nearest_nodes(graph,X_2,Y_2)

start_2 = time.time()
shortest_route_2 = networkx.shortest_path(G=graph,source=closest_nodes_2[0],target=closest_nodes_2[1], weight='t', method='dijkstra')
stop_2 = time.time()

In [130]:
m_dijkstra = folium.Map(location=[43.6374280, -79.3990730], zoom_start=15)

folium.Marker([collection[1], collection[2]], icon = folium.Icon(color='red'), popup="Collection").add_to(m_dijkstra)
folium.Marker([drop1[1], drop1[2]], icon = folium.Icon(color='blue'), popup=drop1[0]).add_to(m_dijkstra)
folium.Marker([drop2[1], drop2[2]], icon = folium.Icon(color='blue'), popup=drop2[0]).add_to(m_dijkstra)

m_dijkstra = osmnx.plot_route_folium(G=graph,route=shortest_route_1, route_map=m_dijkstra)
m_dijkstra = osmnx.plot_route_folium(G=graph,route=shortest_route_2, route_map=m_dijkstra)

folium.Marker([collection[1], collection[2]], icon = folium.Icon(color='red'), popup="Collection").add_to(m_dijkstra)
folium.Marker([drop1[1], drop1[2]], icon = folium.Icon(color='blue'), popup="drop1").add_to(m_dijkstra)
folium.Marker([drop2[1], drop2[2]], icon = folium.Icon(color='blue'), popup="drop2").add_to(m_dijkstra)

m_dijkstra

In [131]:
cost_dij = networkx.path_weight(graph, shortest_route_1, weight='length')
cost_dij += networkx.path_weight(graph, shortest_route_2, weight='length')

running_time_dij = (stop_1-start_1)+(stop_2-start_2)
cost_dij, running_time_dij

(1948.513, 0.02260899543762207)